In [ ]:
import polars as pl
import re
import nltk
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words("english"))
def fast_clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+|[^a-zA-Z\s]", "", text)
    tokens = re.findall(r'\b\w+\b', text)
    return " ".join([t for t in tokens if t not in stop_words])

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import load_dataset
dataset = load_dataset("artem9k/ai-text-detection-pile")["train"]
seen, texts, labels = set(), [], []
for sample in dataset:
    if sample["text"] and sample["text"].strip() and sample["text"] not in seen:
        seen.add(sample["text"])
        texts.append(sample["text"])
        labels.append(1 if sample["source"] == "ai" else 0)

df = pl.DataFrame({"text": texts, "label": labels})
df = df.with_columns(pl.Series("cleaned_text", [fast_clean(t) for t in df["text"]]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df["cleaned_text"].to_list(), df["label"].to_list(),
    test_size=0.2, random_state=42
)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_features=10000, stop_words='english')

X_train_bow = bow_vectorizer.fit_transform(X_train)
X_val_bow = bow_vectorizer.transform(X_val)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [ ]:
def train_and_evaluate(model, name):
    print(f"\nTraining {name}...")
    model.fit(X_train_bow, y_train)
    y_pred = model.predict(X_val_bow)
    print(f"\n{name} Classification Report:")
    print(classification_report(y_val, y_pred))

In [ ]:
train_and_evaluate(LogisticRegression(max_iter=1000), "Logistic Regression")


Training Logistic Regression...

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93    204218
           1       0.84      0.78      0.81     72951

    accuracy                           0.90    277169
   macro avg       0.88      0.86      0.87    277169
weighted avg       0.90      0.90      0.90    277169



In [ ]:
train_and_evaluate(XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1), "XGBoost")


Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:27:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.94      0.91    204218
           1       0.78      0.65      0.71     72951

    accuracy                           0.86    277169
   macro avg       0.83      0.79      0.81    277169
weighted avg       0.86      0.86      0.86    277169



In [ ]:
train_and_evaluate(RandomForestClassifier(n_jobs=-1), "Random Forest")


Training Random Forest...

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.91    204218
           1       0.80      0.67      0.73     72951

    accuracy                           0.87    277169
   macro avg       0.84      0.80      0.82    277169
weighted avg       0.87      0.87      0.87    277169



In [ ]:
! pip install scikit-learn-intelex


In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(norm='l2')
X_train_bow_norm = normalizer.fit_transform(X_train_bow)
X_val_bow_norm = normalizer.transform(X_val_bow)

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=1.0, random_state=42)
svm.fit(X_train_bow_norm, y_train)

y_pred = svm.predict(X_val_bow_norm)
print("SVM Classification Report:")
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


SVM Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95    204218
           1       0.87      0.87      0.87     72951

    accuracy                           0.93    277169
   macro avg       0.91      0.91      0.91    277169
weighted avg       0.93      0.93      0.93    277169

